In [1]:
import skimage.io as io
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from skimage.transform import resize
sample_size=906
n_components=100

In [2]:
label_data=pd.read_csv('labels.csv')
label_data.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


### Resample the Dataset


In [3]:
classes=['maltese_dog','afghan_hound','entlebucher','bernese_mountain_dog','shih-tzu','pomeranian','great_pyrenees','basenji']
label_data=label_data.loc[label_data['breed'].isin(classes)]
label_data.reset_index(drop=True,inplace=True)

In [4]:
le=LabelEncoder()
label_data['breed_no']=le.fit_transform(label_data['breed'])

In [5]:
label_data.head()

,id,breed,breed_no
0,003df8b8a8b05244b1d920bb6cf451f9,basenji,1
1,00693b8bc2470375cc744a6391d397ec,maltese_dog,5
2,0162b3e0144fb1d1ab82fbff3ace9938,shih-tzu,7
3,01ee3c7ff9bcaba9874183135877670e,entlebucher,3
4,021b5a49189665c0442c19b5b33e8cf1,entlebucher,3


In [6]:
label_data['filenames']=label_data['id'].map(lambda x:'train/'+x+'.jpg')

In [7]:
label_data.head()

,id,breed,breed_no,filenames
0,003df8b8a8b05244b1d920bb6cf451f9,basenji,1,train/003df8b8a8b05244b1d920bb6cf451f9.jpg
1,00693b8bc2470375cc744a6391d397ec,maltese_dog,5,train/00693b8bc2470375cc744a6391d397ec.jpg
2,0162b3e0144fb1d1ab82fbff3ace9938,shih-tzu,7,train/0162b3e0144fb1d1ab82fbff3ace9938.jpg
3,01ee3c7ff9bcaba9874183135877670e,entlebucher,3,train/01ee3c7ff9bcaba9874183135877670e.jpg
4,021b5a49189665c0442c19b5b33e8cf1,entlebucher,3,train/021b5a49189665c0442c19b5b33e8cf1.jpg


In [8]:
label_data['breed'].describe()

count             906
unique              8
top       maltese_dog
freq              117
Name: breed, dtype: object

In [9]:
image_data=io.imread_collection(label_data['filenames'])

In [10]:
len(image_data)

906

In [11]:
labels=np.array(label_data.breed_no)

In [12]:
labels=labels[:sample_size]

In [13]:
resized_images=[]
for i in image_data[:sample_size]:
    resized_images.append(resize(i,(200,200)).ravel())

C:\Users\deban\AppData\Local\Continuum\anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [14]:
training=np.array(resized_images)

In [15]:
training.shape

(906, 120000)

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(training, labels,test_size=0.25, random_state=42)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca=PCA(n_components=n_components,whiten=True).fit(X_train)

In [ ]:
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [ ]:
param_grid = {
         'C': [1e3, 5e3, 1e4, 5e4, 1e5],
          'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
          }

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [ ]:
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid)
clf = clf.fit(X_train_pca, y_train)

In [ ]:
clf.predict(X_test_pca[:5])

In [ ]:
X_train_pca.shape

In [ ]:
clf.score(X_test_pca,y_test)

In [ ]:
labels.shape

In [ ]:
training.shape